This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally,
you'll need to install [RISE](https://rise.readthedocs.io/).

But it's probably simpler to just visit the
[online version](https://nbviewer.jupyter.org/format/slides/github/mgeier/splines2021/blob/main/presentation.ipynb).

# Rotation Splines for Spatial Audio Scenes

Matthias Geier

https://github.com/mgeier/splines2021

## Audio Scene Description Format (ASDF)

* https://AudioSceneDescriptionFormat.readthedocs.io/
* https://github.com/AudioSceneDescriptionFormat/asdf-rust
* https://github.com/SoundScapeRenderer/ssr/pull/155
* splines for sound source trajectories
* splines for rotation
  * source rotations
  * rotation of groups of sources
  * rotation of local coordinate systems
* splines for volume envelopes
* everything is still *work in progress*!

## Overview

* splines in Euclidean space
  * Bézier splines
  * De Casteljau's algorithm
  * Catmull–Rom splines
* three-dimensional rotation
* unit quaternions
  * Slerp (**s**pherical **l**inear int**erp**olation)
* rotation splines
  * De Casteljau with Slerp
  * Catmull–Rom with Slerp

## Splines in Euclidean Space

* https://splines.readthedocs.io/
* https://splines.readthedocs.io/euclidean/

## Parametric Piecewise Cubic Polynomial Curves

* $\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i$
  * univariate
  * $n$-dimensional
  * uniform vs. non-uniform

## Bézier Splines

* most common: *cubic* Bézier splines
* De Casteljau's algorithm
  * multiple stages, each only using **l**inear int**erp**olation
    * $\operatorname{Lerp}(\boldsymbol{x}_0, \boldsymbol{x}_1 ;t) = (1 - t) \boldsymbol{x}_0 + t \boldsymbol{x}_1$,
      where $0 \le t \le 1$
  * https://splines.readthedocs.io/euclidean/bezier-de-casteljau.html

## Catmull–Rom Splines

* https://splines.readthedocs.io/euclidean/catmull-rom-properties.html

## Three-dimensional Rotation

* azimuth, elevation, roll
  * order matters!
  * gimbal lock
* axis and angle
* rotation matrix
* unit quaternion
* ... and more: homogeneous coordinates, geometric algebra, etc.

## Quaternions

* algebraic: $w + x\mathbf{i} + y\mathbf{j} + z\mathbf{k}$
  * $\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{ijk} = -1$
  * $\mathbf{ij} = \mathbf{k}$
  * $\mathbf{ji} = -\mathbf{k}$
* scalar and vector: $(w, \vec{v}) = (w, (x, y, z))$
  * "real" and "imaginary"
* 4D vector space (with additional operations)
  * $(w, x, y, z)$ vs. $(x, y, z, w)$
* ... and more: 4x4 real matrix, 2x2 complex matrix, etc.

## Unit Quaternions $\Leftrightarrow$ Rotations

* $q_i = (\cos \frac{\alpha_i}{2}, \vec{n}_i \sin \frac{\alpha_i}{2})$
  * (normalized) rotation axis $\vec{n}_i$
  * angle $\alpha_i$ (in radians)
* double cover
  * $q$ and $-q$ represent the same rotation!

## Unit Quaternion Operations

* quaternion multiplication: $q_1 q_0$
  * rotation $q_0$ followed by rotation $q_1$ (read from right to left)
  * $q_0 q_1 \ne q_1 q_0$ (except if the rotation axis is the same)
* identity: $(1, (0, 0, 0))$
* inverse: $q^{-1}$
  * same rotation axis, negated angle
  * $q q^{-1} = q^{-1} q = (1, (0, 0, 0))$

## Slerp, part 1

* **s**pherical **l**inear int**erp**olation: $\operatorname{Slerp}(q_0, q_1; t)$, where $0 \le t \le 1$
* intermediate step: change angle linearly from $0$ to $\alpha$
  * reminder (axis/angle):
    $q = \left(\cos \frac{\alpha}{2}, \vec{n} \sin \frac{\alpha}{2}\right)$
  * rotation axis $\vec{n}$ stays constant
  * if $\alpha = 0 \to q = \boldsymbol{1} = (1, (0, 0, 0))$
  * $\operatorname{Slerp}(\boldsymbol{1}, q; t) = \left(\cos \frac{\alpha t}{2}, \vec{n} \sin \frac{\alpha t}{2}\right) = q^t$
* $\operatorname{Slerp}(q_0, q_1; t) = \operatorname{Slerp}(\boldsymbol{1}, q_{0,1}; t) \, q_0$
  * $q_{0,1}$: relative rotation from $q_0$ to $q_1$

## Relative Rotation

* $q_{0,1}$: rotation from $q_0$ to $q_1$

\begin{equation*}
q_{0,1} q_0 = q_1
\end{equation*}

* right-multiply both sides with ${q_0}^{-1}$

\begin{equation*}
q_{0,1} q_0 {q_0}^{-1} = q_1 {q_0}^{-1}
\end{equation*}

* $q_0 {q_0}^{-1}$ cancels out

\begin{equation*}
q_{0,1} = q_1 {q_0}^{-1}
\end{equation*}

## Slerp, part 2

* $\operatorname{Slerp}(q_0, q_1; t) = \operatorname{Slerp}(\boldsymbol{1}, q_{0,1}; t) \, q_0$
  * $\operatorname{Slerp}(\boldsymbol{1}, q; t) = q^t$

  * $q_{0,1} = q_1 {q_0}^{-1}$

* $\operatorname{Slerp}(q_0, q_1; t) = \left(q_1 {q_0}^{-1}\right)^t \, q_0$
* https://splines.readthedocs.io/rotation/slerp.html

## Rotation Splines

* https://splines.readthedocs.io/rotation/de-casteljau.html
* https://splines.readthedocs.io/rotation/catmull-rom-uniform.html
* https://splines.readthedocs.io/rotation/catmull-rom-non-uniform.html